# Thompson Sampling with Pluggable Stopping Criteria

This notebook demonstrates Thompson Sampling for Multi-Armed Bandits with different stopping criteria:
1. Fixed Trials (Business Constraints)
2. Confidence Threshold (Bayesian)
3. Convergence-Based (Traffic Allocation Stability)

## Setup: Import Libraries

In [ ]:
import numpy as np
from scipy.stats import beta
import matplotlib.pyplot as plt

## Helper Function: Check Minimum Pulls

In [ ]:
def check_minimum_pulls(alpha_params, beta_params, min_pulls):
    """Helper: Check if all arms have been pulled at least min_pulls times."""
    pull_counts = (alpha_params + beta_params) - 2  # Subtract initial (1,1)
    return np.all(pull_counts >= min_pulls)

In [ ]:
def plot_final_distributions(alpha_params, beta_params, true_rates, best_arm_index, save_file=False):
    """Visualize the final Beta distribution for all arms.
    
    Args:
        alpha_params: Final alpha parameters for each arm
        beta_params: Final beta parameters for each arm
        true_rates: Ground truth conversion rates
        best_arm_index: Index of the best arm
    """
    n_arms = len(alpha_params)
    x = np.linspace(0, 0.4, 1000)
    
    plt.figure(figsize=(12, 6))
    colors = plt.cm.Set2(np.linspace(0, 1, n_arms))
    
    for i in range(n_arms):
        # Calculate Beta distribution PDF
        y = beta.pdf(x, alpha_params[i], beta_params[i])
        
        # Mark the best arm with thicker line
        linewidth = 3 if i == best_arm_index else 2
        linestyle = '-' if i == best_arm_index else '--'
        
        label = f'Arm {i+1}'
        if i == best_arm_index:
            label += ' 👑 (best)'
        label += f' | α={alpha_params[i]:.0f}, β={beta_params[i]:.0f}'
        
        plt.plot(x, y, label=label, linewidth=linewidth, 
                linestyle=linestyle, color=colors[i])
        
        # Mark true conversion rate with vertical line (only if within range)
        if true_rates[i] <= 0.4:
            plt.axvline(true_rates[i], color=colors[i], 
                       linestyle=':', alpha=0.5, linewidth=1)
    
    plt.xlabel('Conversion Rate', fontsize=12)
    plt.ylabel('Probability Density', fontsize=12)
    plt.title('Final Beta Distributions for All Arms\n(Dotted lines show true rates)', 
             fontsize=14, fontweight='bold')
    plt.legend(loc='upper right', fontsize=9)
    plt.grid(True, alpha=0.3)
    plt.xlim(0, 0.4)
    plt.tight_layout()
    
    if save_file:
        # Save figure
        import os
        os.makedirs('../Images', exist_ok=True)
        plt.savefig('../Images/ch15_final_beta_distributions.png', dpi=150, bbox_inches='tight')
    
    plt.show()


def plot_evolution_of_best_arm(history_snapshots, best_arm_index, true_rate, save_file=False):
    """Visualize how the Beta distribution of the best arm evolves over time.
    
    Args:
        history_snapshots: List of (trial, alpha, beta) tuples for the best arm
        best_arm_index: Index of the best arm
        true_rate: True conversion rate of the best arm
    """
    x = np.linspace(0, 0.4, 1000)
    
    plt.figure(figsize=(12, 6))
    colors = plt.cm.viridis(np.linspace(0, 1, len(history_snapshots)))
    
    for idx, (trial, alpha, beta_param) in enumerate(history_snapshots):
        y = beta.pdf(x, alpha, beta_param)
        
        # First snapshot uses thicker line
        linewidth = 3 if idx == 0 else 2
        alpha_val = 0.4 if idx == 0 else 0.7
        
        label = f'Trial {trial}: α={alpha:.0f}, β={beta_param:.0f}'
        plt.plot(x, y, label=label, linewidth=linewidth, 
                alpha=alpha_val, color=colors[idx])
    
    # Mark true conversion rate (only if within range)
    if true_rate <= 0.4:
        plt.axvline(true_rate, color='red', linestyle='--', 
                   linewidth=2, label=f'True rate ({true_rate:.2%})', alpha=0.8)
    
    plt.xlabel('Conversion Rate', fontsize=12)
    plt.ylabel('Probability Density', fontsize=12)
    plt.title(f'Evolution of Beta Distribution for Arm {best_arm_index + 1} (Best Arm)', 
             fontsize=14, fontweight='bold')
    plt.legend(loc='upper right', fontsize=9)
    plt.grid(True, alpha=0.3)
    plt.xlim(0, 0.4)
    plt.tight_layout()
    
    if save_file:
        # Save figure
        import os
        os.makedirs('../Images', exist_ok=True)
        plt.savefig('../Images/ch15_evolution_best_arm.png', dpi=150, bbox_inches='tight')
    
    plt.show()

## Visualization Functions

## Stopping Criterion 1: Fixed Trials

In [ ]:
def fixed_trials_stop(trial, alpha_params, beta_params, max_trials=1000, min_pulls=50, **kwargs):
    """Stop after a fixed number of trials (Business Constraints).
    
    Requires minimum pulls per arm before stopping.
    """
    if not check_minimum_pulls(alpha_params, beta_params, min_pulls):
        return False
    return trial >= max_trials

## Stopping Criterion 2: Confidence Threshold

In [ ]:
def confidence_threshold_stop(trial, alpha_params, beta_params, confidence=0.95, 
                              min_pulls=50, n_samples=10000, check_every=50, 
                              verbose=False, **kwargs):
    """Stop when posterior probability of best arm exceeds threshold (Bayesian Confidence).
    
    Requires minimum pulls per arm to ensure sufficient data for reliable inference.
    Only checks confidence every 'check_every' trials to reduce computational cost.
    """
    # Ensure minimum data collection first
    if not check_minimum_pulls(alpha_params, beta_params, min_pulls):
        return False
    
    # Only check confidence periodically (expensive Monte Carlo calculation)
    # Check on trial 0, then every check_every trials after min_pulls requirement is met
    if trial % check_every != 0:
        return False
    
    # Sample from each arm's posterior distribution
    # samples[i, j] = j-th sample from arm i's Beta(alpha_i, beta_i) distribution
    n_arms = len(alpha_params)
    samples = np.array([np.random.beta(alpha_params[i], beta_params[i], n_samples) 
                        for i in range(n_arms)])
    
    # For each of the n_samples trials, which arm had the highest sampled rate?
    # best_arm_of_trials[j] = index of arm with highest sample in trial j
    # This tells us which arm "won" each simulated comparison
    best_arm_of_trials = np.argmax(samples, axis=0)
    
    # Count how many times each arm won across all n_samples trials
    # probabilities[i] = fraction of trials where arm i had the highest sample
    # This estimates P(arm i is best | observed data)
    # 
    # IMPORTANT: These probabilities already account for sample size uncertainty!
    # An arm with few samples has a wide Beta distribution, so even if its 
    # empirical rate is high, it won't consistently win the Monte Carlo trials.
    # Example: Arm A (100 pulls, 15% rate) vs Arm B (10 pulls, 20% rate)
    #   - Arm B has higher observed rate, BUT high uncertainty
    #   - Arm A's narrow distribution might still win more Monte Carlo trials
    #   - So probabilities[A] could be > probabilities[B]
    probabilities = np.bincount(best_arm_of_trials, minlength=n_arms) / n_samples
    
    max_prob = np.max(probabilities)
    
    # Optional: Print diagnostic info
    if verbose and trial % (check_every * 10) == 0:
        best_idx = np.argmax(probabilities)
        print(f"  Trial {trial}: P(Arm {best_idx+1} is best) = {max_prob:.2%}, "
              f"Top 3 probs: {sorted(probabilities, reverse=True)[:3]}")
    
    # Stop if we're confident enough about which arm is best
    # The arm with max(probabilities) is our best guess given the data AND uncertainty
    # If max(probabilities) > 0.95, we're 95% confident one arm is truly best
    return max_prob > confidence

## Stopping Criterion 3: Convergence

In [ ]:
def convergence_stop(trial, arms_chosen, alpha_params, beta_params, 
                    window_size=100, threshold=0.80, min_pulls=50, **kwargs):
    """Stop when leading arm gets >threshold% of traffic in recent window.
    
    Requires minimum pulls per arm to avoid premature convergence from early luck.
    """
    # Ensure minimum data collection first
    if not check_minimum_pulls(alpha_params, beta_params, min_pulls):
        return False
        
    if trial < window_size:
        return False
    
    recent_choices = arms_chosen[-window_size:]
    most_common_count = np.max(np.bincount(recent_choices))
    return (most_common_count / window_size) > threshold

## Thompson Sampling Simulation Function

In [ ]:
def simulate_thompson_sampling(true_conversion_rates, stopping_criterion, max_trials=10000, 
                               track_history=False, **stop_params):
    """
    Run Thompson Sampling with a pluggable stopping criterion.
    
    Args:
        true_conversion_rates: List of true conversion rates for each arm
        stopping_criterion: Function that returns True when experiment should stop
        max_trials: Safety limit on maximum trials
        track_history: If True, track evolution of best arm's distribution
        **stop_params: Additional parameters passed to stopping_criterion
        
    Returns:
        dict: Contains final parameters, best arm index, and optionally history snapshots
    """
    n_arms = len(true_conversion_rates)
    best_arm_index = np.argmax(true_conversion_rates)
    
    # Initialize Beta distribution parameters (uniform prior)
    alpha_params = np.ones(n_arms)
    beta_params = np.ones(n_arms)
    
    # Track history
    arms_chosen = []
    total_rewards = 0
    stopping_criterion_met = False
    
    # Track evolution of best arm's distribution (uniformly sample 5 snapshots)
    history_snapshots = []
    if track_history:
        # Always include initial state
        history_snapshots.append((0, alpha_params[best_arm_index], beta_params[best_arm_index]))

    for trial in range(max_trials):
        # 1. Sample from each arm's posterior distribution
        sampled_rates = [np.random.beta(alpha_params[i], beta_params[i]) 
                        for i in range(n_arms)]
        
        # 2. Choose arm with highest sample
        chosen_arm = np.argmax(sampled_rates)
        arms_chosen.append(chosen_arm)
        
        # 3. Simulate reward
        reward = 1 if np.random.rand() < true_conversion_rates[chosen_arm] else 0
        total_rewards += reward
        
        # 4. Update posterior
        if reward == 1:
            alpha_params[chosen_arm] += 1
        else:
            beta_params[chosen_arm] += 1
        
        # 5. Check stopping criterion
        if stopping_criterion(
            trial=trial,
            alpha_params=alpha_params,
            beta_params=beta_params,
            arms_chosen=arms_chosen,
            **stop_params
        ):
            stopping_criterion_met = True
            break
    
    # Capture history snapshots uniformly (4 intermediate + 1 final)
    if track_history and trial > 0:
        final_trial = trial
        # Select 4 intermediate points uniformly
        snapshot_trials = np.linspace(0, final_trial, 5, dtype=int)[1:-1]
        
        # Re-run to capture snapshots (quick since we know the sequence)
        alpha_temp = np.ones(n_arms)
        beta_temp = np.ones(n_arms)
        snapshot_idx = 0
        
        for t in range(final_trial + 1):
            if t > 0:  # Skip trial 0 (already captured)
                chosen = arms_chosen[t-1]
                reward = 1 if np.random.rand() < true_conversion_rates[chosen] else 0
                if reward == 1:
                    alpha_temp[chosen] += 1
                else:
                    beta_temp[chosen] += 1
                
                # Capture snapshot at predetermined trials
                if snapshot_idx < len(snapshot_trials) and t == snapshot_trials[snapshot_idx]:
                    history_snapshots.append((t, alpha_temp[best_arm_index], beta_temp[best_arm_index]))
                    snapshot_idx += 1
        
        # Always include final state
        history_snapshots.append((final_trial, alpha_params[best_arm_index], beta_params[best_arm_index]))
    
    # Report results
    estimated_rates = alpha_params / (alpha_params + beta_params)
    pull_counts = np.bincount(arms_chosen, minlength=n_arms)
    discovered_best_arm = np.argmax(pull_counts)
    
    if stopping_criterion_met:
        print(f"Stopped after {trial + 1} trials using {stopping_criterion.__name__}")
    else:
        print(f"Stopped after {trial + 1} trials as max_trial reached.")
    print(f"Total rewards: {total_rewards}\n")
    print(f"Ground Truth: Arm {best_arm_index + 1} is best (true rate: {true_conversion_rates[best_arm_index]:.2%})")
    print(f"Discovered:   Arm {discovered_best_arm + 1} chosen most (estimated rate: {estimated_rates[discovered_best_arm]:.2%})")
    
    if discovered_best_arm == best_arm_index:
        print("✓ Algorithm correctly identified the best arm!\n")
    else:
        print("✗ Algorithm converged to a suboptimal arm\n")
    
    print("Results by arm:")
    for i in range(n_arms):
        marker = "👑" if i == best_arm_index else "  "
        chosen_marker = "←" if i == discovered_best_arm else " "
        print(f"{marker} Arm {i+1}: {pull_counts[i]:4d} pulls, "
              f"estimated rate: {estimated_rates[i]:.2%} {chosen_marker}")
    print()
    
    # Return data for visualization
    return {
        'alpha_params': alpha_params,
        'beta_params': beta_params,
        'best_arm_index': best_arm_index,
        'true_rates': true_conversion_rates,
        'history_snapshots': history_snapshots if track_history else None
    }

## Example Usage: Run All Three Stopping Criteria

In [ ]:
# Define true conversion rates for 5 arms
# Arm 3 is the best with 11% conversion rate
true_rates = [0.05, 0.08, 0.11, 0.09, 0.07]

# Set random seed for reproducibility
np.random.seed(42)

### Test 1: Fixed Trials Criterion

In [ ]:
print("=" * 70)
print("Criterion 1: Fixed Trials (1000 max, 50 min per arm)")
print("=" * 70)
# Pass max_trials=1000 which serves both as the stopping criterion AND the loop limit
result = simulate_thompson_sampling(true_rates, fixed_trials_stop, 
                                   max_trials=5000, min_pulls=50, track_history=True)

# Visualize final distributions
plot_final_distributions(result['alpha_params'], result['beta_params'], 
                         result['true_rates'], result['best_arm_index'])

# Visualize evolution of best arm
if result['history_snapshots']:
    plot_evolution_of_best_arm(result['history_snapshots'], 
                               result['best_arm_index'], 
                               result['true_rates'][result['best_arm_index']])

### Test 2: Convergence Criterion

In [ ]:
print("=" * 70)
print("Criterion 2: Convergence (80% in window, 200 min per arm)")
print("=" * 70)
result = simulate_thompson_sampling(true_rates, convergence_stop, 
                                   window_size=100, threshold=0.75, 
                                   min_pulls=200, max_trials=10000, track_history=True)

# Visualize final distributions
plot_final_distributions(result['alpha_params'], result['beta_params'], 
                         result['true_rates'], result['best_arm_index'])

# Visualize evolution of best arm
if result['history_snapshots']:
    plot_evolution_of_best_arm(result['history_snapshots'], 
                               result['best_arm_index'], 
                               result['true_rates'][result['best_arm_index']])

### Test 3: Confidence Threshold Criterion

### Test 3: Confidence Threshold Criterion

**Note on Instability:** The confidence threshold criterion can be unstable when:
- Arms have small differences (11% vs 10% here)
- Conversion rates are low (5-11%), requiring many samples
- Multiple arms are competitive

This is realistic behavior - in production, you'd either:
1. Lower the confidence threshold (e.g., 80-85%)
2. Increase max_trials budget
3. Combine with other criteria (e.g., max trials OR confidence)
4. Use wider arm differences for testing

In [ ]:
print("=" * 70)
print("Criterion 3: Confidence Threshold (85%, 50 min per arm, check every 50 trials)")
print("=" * 70)
# Reset random seed for reproducibility across notebook runs
np.random.seed(42)
result = simulate_thompson_sampling(true_rates, confidence_threshold_stop, 
                                   confidence=0.85, min_pulls=50, check_every=50, 
                                   verbose=True, max_trials=10000, track_history=True)

# Visualize final distributions
plot_final_distributions(result['alpha_params'], result['beta_params'], 
                         result['true_rates'], result['best_arm_index'])

# Visualize evolution of best arm
if result['history_snapshots']:
    plot_evolution_of_best_arm(result['history_snapshots'], 
                               result['best_arm_index'], 
                               result['true_rates'][result['best_arm_index']])

In [ ]:
# Test again with very close arms
true_rates = [0.05, 0.08, 0.11, 0.10, 0.07]

# Set random seed for reproducibility
np.random.seed(42)

print("=" * 70)
print("Criterion 2: Convergence (80% in window, 200 min per arm)")
print("=" * 70)
result = simulate_thompson_sampling(true_rates, convergence_stop, 
                                   window_size=100, threshold=0.75, 
                                   min_pulls=200, max_trials=10000, track_history=True)

# Visualize final distributions
plot_final_distributions(result['alpha_params'], result['beta_params'], 
                         result['true_rates'], result['best_arm_index'])

# Visualize evolution of best arm
if result['history_snapshots']:
    plot_evolution_of_best_arm(result['history_snapshots'], 
                               result['best_arm_index'], 
                               result['true_rates'][result['best_arm_index']])